## I will present iYli21
### A genome-scale metabolic model

### Packages

In [1]:
import cobra
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

import pandas as pd

In [3]:
## Set variables 
pathway = "./iYli21_v1.xml"


In [4]:
# read model
model = read_sbml_model(pathway)


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [5]:
# investigate number of metabolites, reactions and genes
modelName = model.name
reacNum = len(model.reactions)
metaboNum = len(model.metabolites)
geneNum = len(model.genes)

print('The {name} has {reacNum} reactions, {metaboNum} metabolites and {geneNum} genes'.format(name=modelName, reacNum=reacNum, metaboNum=metaboNum, geneNum=geneNum))

The Genome-scale model of Yarrowia lipolytica W29 has 2285 reactions, 1868 metabolites and 1058 genes


### Overview about the genes, metabolites and reaction names

In [6]:
# iterate over genes and build panda data frame
print(type(model))
# print(model.solver)
model.solver = 'glpk'
print(model.summary())
print(type(model.genes))
print(type(model.metabolites))
print(type(model.reactions))



<class 'cobra.core.model.Model'>
Objective
1.0 biomass_C = 0.2827037239904458

Uptake
------
 Metabolite Reaction    Flux  C-Number C-Flux
m1031[C_ex]    R1016    1.91         0  0.00%
 m511[C_ex]    R1070    2.43         0  0.00%
 m342[C_ex]    R1164   3.875         0  0.00%
m1339[C_ex]    R1287   5.057         0  0.00%
 m215[C_ex]    R1296 0.01286         0  0.00%
m1113[C_ex]    R1333 0.01919         0  0.00%

Secretion
---------
 Metabolite Reaction    Flux  C-Number C-Flux
m1176[C_ex]    R1030  -5.042         0  0.00%
 m214[C_ex]    R1364  -16.63         0  0.00%
m1401[C_cy]    R1373 -0.2827         0  0.00%

<class 'cobra.core.dictlist.DictList'>
<class 'cobra.core.dictlist.DictList'>
<class 'cobra.core.dictlist.DictList'>


### investigate glucose exchange reaction
How to iterate metabolites of a reaction

In [7]:
glu_ex = model.reactions.get_by_id('R851')
glu_ex.metabolites
glu_metabolites = []
for metabol, coef in glu_ex.metabolites.items():
    elem1 = model.metabolites.get_by_id(metabol.id)
    glu_metabolites.append(elem1)
substrate = glu_metabolites[0]
product = glu_metabolites[1]
# print(substrate.summary())
# print(product.summary())
print(substrate.id)
print(substrate.name)

print(product.id)
print(product.name)

print("Bounds: ", glu_ex.bounds)



m511[C_ex]
D-glucose_C6H12O6
m295[C_cy]
D-glucose_C6H12O6
Bounds:  (0.0, 1000.0)


### investigate reaction

### FBA: testing without changes

In [8]:
solution = model.optimize()
print(solution)
print(solution.status)
fluxPD = solution.fluxes

fluxPD.to_csv('./output/tmp/fluxPD.csv')

<Solution 0.283 at 0x7fc0097c15e0>
optimal


### Memote returned errors in annotation + stoichimetric consistency + mass balance


In [9]:
glu_transport = model.reactions.get_by_id("R41")
# print(glu_transport.gpr)
glu_transport.metabolites
m112 = model.metabolites.get_by_id("m112[C_cy]")
m112
m32 = model.metabolites.get_by_id("m32[C_cy]")
m32




Metabolite identifier,m32[C_cy]
Name,H2O_H2O
Memory address,0x7fc001a795e0
Formula,
Compartment,C_cy
In 320 reaction(s),"R2067, R405, R2074, R172, R2099, R2226, R2229, R568, R1393, R1760, biomass_C, R69, R505, R652, R985, R341, R262, R2197, R601, R68, R702, R1474, R536, R10, R2201, R526, R703, R369, R651, R389,..."


### Build table of exchange reactions (to find the carbon sources)

In [10]:
exchange_members = model.groups.group22.members
type(exchange_members)
exchange_list = []
for key in exchange_members:
    exchange_list.append(key)

tmp = exchange_list[0]
tmp

exchange_ids = [x.id for x in exchange_list]
exchange_names = [x.name for x in exchange_list]
exchange_lb = [x.lower_bound for x in exchange_list]
exchange_ub = [x.upper_bound for x in exchange_list]

zipped = list(zip(exchange_ids, exchange_names, exchange_lb, exchange_ub))
# generate pd dataframe # id, name, lb, ub
exchangeDF = pd.DataFrame(zipped, columns=['id', 'name', 'lb', 'ub'])
exchangeDF.to_csv('./output/exchangeReactions.csv', index=False)
# I have found the set of reactions of the group exchange

### In Guo et al. they used 0.61, 0.64 and 2.43 mmol·gDCW−1·h−1
#### Very important: check if every carbon source is set to 0

#### Helperfunctions

In [11]:
# clip "exchange" from exchange reaction name to get the carbon source name
def getCarbonName(reaction): # assuming naming convention: 'carbonName exchange' for each carbon source
    reactionName = reaction.name
    nameList = reactionName.split(' ')
    if (str(nameList[-1]) != 'exchange'):
        print('Error in getCarbonName, assumption not valid')
    return ' '.join(nameList[:-1])

# Function to print feeding results
def printFeedingResult(reaction, objectiveValue, objectiveFunction = 'Growth'):
    carbonSource = getCarbonName(reaction)
    lowerBound = reaction.lower_bound
    uptake = lowerBound * -1 # assuming feeding is based on lower_bound which has a minus as first charakter
    print('The calculated objective value for {objectiveFunction} is {objectiveValue}. This result was obtained by uptake of {uptake} {carbonSource}'.format(objectiveFunction=objectiveFunction, objectiveValue=objectiveValue,uptake=uptake, carbonSource=carbonSource))

In [12]:
# from exchangeReactions.csv: R1070 is D-glucose exchange
gluExchange = model.reactions.get_by_id("R1070")
gluExchange
# set lower bound to -0.61
gluExchange.bounds = (-0.61, 1000)
glu061 = model.optimize()
glu061.objective_value # result: 0.03133; literature: 0.031
printFeedingResult(gluExchange, glu061.objective_value)
# set lower bound to -0.64
gluExchange.bounds = (-0.64, 1000)
glu064 = model.optimize()
glu064.objective_value # result: 0.03548; literature: 0.036
printFeedingResult(gluExchange, glu064.objective_value)
gluExchange.bounds = (-2.43, 1000)
glu243 = model.optimize()
glu243.objective_value # result: 0.2827; literature: 0.28
printFeedingResult(gluExchange, glu243.objective_value)

The calculated objective value for Growth is 0.03133039072588739. This result was obtained by uptake of 0.61 D-glucose
The calculated objective value for Growth is 0.035480781742887386. This result was obtained by uptake of 0.64 D-glucose
The calculated objective value for Growth is 0.282703723990442. This result was obtained by uptake of 2.43 D-glucose


### Simulations of other carbon sources 

In [13]:
# set glucose uptake to zero:
gluExchange.bounds = (0, 0)

# D-Arabinose exchange R1062
dArabinose = model.reactions.get_by_id('R1062')
# set D-Arabinose uptake to 2.43
dArabinose.bounds = (-2.43, 1000)
dArabResult = model.optimize()
dArabResult.objective_value # result: 0.2827; literature: 0.28
printFeedingResult(dArabinose, dArabResult.objective_value)
dArabinose.bounds = (0, 0)

The calculated objective value for Growth is 0.22685710664831604. This result was obtained by uptake of 2.43 D-arabinose


### Table of metabolism type and corresponding genes

### Playground for reaction components
searching for i-Erythritol as carbon source

In [14]:
# R1994,(R)-3-Hydroxybutanoate exchange,0.0,1000.0
R1994 = model.reactions.get_by_id('R1994')
R1994.metabolites #m1849[C_ex]
m1849 = model.metabolites.get_by_id('m1849[C_ex]')

m1849

# R947,"(R,R)-2,3-butanediol exchange",0.0,0.0
R947 = model.reactions.get_by_id('R947')
R947.metabolites #m1020[C_ex]
m1020 = model.metabolites.get_by_id('m1020[C_ex]')

m1020

# R1390,"ergosta-5,7,22,24(28)-tetraen-3beta-ol exchange",0.0,0.0
R1390 = model.reactions.get_by_id('R1390')
R1390.metabolites #m1407[C_ex]
m1407 = model.metabolites.get_by_id('m1407[C_ex]')

m1407

Metabolite identifier,m1407[C_ex]
Name,"ergosta-5,7,22,24(28)-tetraen-3beta-ol_C28H42O"
Memory address,0x7fc020f67fd0
Formula,
Compartment,C_ex
In 2 reaction(s),"R1390, R1391"


searching for gamma-aminobutyrate

In [15]:
# R1134,gamma-aminobutyrate exchange,0.0,0.0

R1134 = model.reactions.get_by_id('R1134')
R1134 # m1021[C_ex]
m1021 = model.metabolites.get_by_id('m1021[C_ex]')
m1021 # gamma-aminobutyrate_C4H9NO2


Metabolite identifier,m1021[C_ex]
Name,gamma-aminobutyrate_C4H9NO2
Memory address,0x7fc020f4a1f0
Formula,
Compartment,C_ex
In 2 reaction(s),"R809, R1134"


searching for B-Hydroxy-Butyric acid

In [16]:
R1994 = model.reactions.get_by_id('R1994')
R1994 # m1849[C_ex]


Reaction identifier,R1994
Name,(R)-3-Hydroxybutanoate exchange
Memory address,0x7fc033e3cfa0
Stoichiometry,m1849[C_ex] --> (R)-3-Hydroxybutanoate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


searching for L-Malic acid

In [17]:
# R949,(S)-malate exchange,0.0,0.0
R949 = model.reactions.get_by_id('R949')
R949 #C4H6O5 -> L-Malic acid

Reaction identifier,R949
Name,(S)-malate exchange
Memory address,0x7fc033d62460
Stoichiometry,m1130[C_ex] --> (S)-malate_C4H6O5 -->
GPR,
Lower bound,0.0
Upper bound,0.0


searching for fumaric acid

In [18]:
# R1133,fumarate(2-) exchange,0.0,0.0
R1133 = model.reactions.get_by_id('R1133')
R1133 # C4H4O4: Fumaric acid (PubChem) # fumerate 2- is the ion (in solution)

Reaction identifier,R1133
Name,fumarate(2-) exchange
Memory address,0x7fc021240d30
Stoichiometry,m1233[C_ex] --> fumarate_C4H4O4 -->
GPR,
Lower bound,0.0
Upper bound,0.0


searching for citric acid

In [19]:
# R1045,citrate(3-) exchange,0.0,1000.0
# If C6H8O7 it is citric acid
R1045 = model.reactions.get_by_id('R1045')
R1045 # -> it is citric acid

Reaction identifier,R1045
Name,citrate(3-) exchange
Memory address,0x7fc033dcf3a0
Stoichiometry,m1187[C_ex] --> citrate_C6H8O7 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


searching for alpha-Ketoglutaric acid

In [20]:
# R964,2-oxoglutarate exchange,0.0,0.0
R964 = model.reactions.get_by_id('R964')
R964 # C5H6O5 -> alpha-Ketoglutaric acid

Reaction identifier,R964
Name,2-oxoglutarate exchange
Memory address,0x7fc033d9a8b0
Stoichiometry,m1145[C_ex] --> 2-oxoglutarate_C5H6O5 -->
GPR,
Lower bound,0.0
Upper bound,0.0


searching for acetic acid

In [21]:
# R999,acetate exchange,0.0,0.0
R999 = model.reactions.get_by_id('R999')
R999 # C2H4O2 -> could be

Reaction identifier,R999
Name,acetate exchange
Memory address,0x7fc033db9520
Stoichiometry,m1025[C_ex] --> acetate_C2H4O2 -->
GPR,
Lower bound,0.0
Upper bound,0.0


searching for D-Mannitol

In [22]:
# R1965 name="mannitol transport extracellular"
R1965 = model.reactions.get_by_id('R1965')
R1965

# mannitol = model.metabolites.get_by_id('m1804[C_cy]')
# mannitol

Reaction identifier,R1965
Name,mannitol transport extracellular
Memory address,0x7fc033e1ed00
Stoichiometry,m1804[C_cy] <=> m1824[C_ex] D-mannitol_ <=> D-mannitol_
GPR,
Lower bound,-1000.0
Upper bound,1000.0


searching for glycerol

In [23]:
# R856,glycerol transport,False,"['H+_p+1', 'glycerol_C3H8O3', 'H+_p+1', 'glycerol_C3H8O3']","['m342[C_ex]', 'm1056[C_ex]', 'm10[C_cy]', 'm277[C_cy]']"
R856 = model.reactions.get_by_id('R856')
R856 # inwards 
# R857,glycerol transport via channel,False,"['glycerol_C3H8O3', 'glycerol_C3H8O3']","['m277[C_cy]', 'm1056[C_ex]']"
R857 = model.reactions.get_by_id('R857')
R857 # outwards


Reaction identifier,R857
Name,glycerol transport via channel
Memory address,0x7fc02118b4f0
Stoichiometry,m277[C_cy] --> m1056[C_ex] glycerol_C3H8O3 --> glycerol_C3H8O3
GPR,YALI1F00616g
Lower bound,0.0
Upper bound,1000.0


### Trying to understand (R)- and (S)-lactate exchange
R944,(R)-lactate exchange,0.0,1000.0
Which metabolite is this? D- L-?
https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:422 -> (S)-lactic acid = L-lactic acid
https://pubchem.ncbi.nlm.nih.gov/compound/%28R%29-lactic%20acid -> (R)-lactic acid = D-lactic acid

In [24]:
R944 = model.reactions.get_by_id('R944')
R944 #(R)-lactate exchange, metabolite: m1043[C_ex] / (R)-lactate_C3H6O3


Reaction identifier,R944
Name,(R)-lactate exchange
Memory address,0x7fc001b5cb20
Stoichiometry,m1043[C_ex] --> (R)-lactate_C3H6O3 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Understanding of EXC_OUT_* reactions (3)
?EXC_OUT_m1803 = R1900,isocitrate_C6H8O7 exchange??
?EXC_OUT_m1826 = R1966,erythritol_ exchange??
?EXC_OUT_m1824 = R1967,D-mannitol_ exchange??

In [25]:
# R1966, ?EXC_OUT_m1826 = R1966,erythritol_ exchange??
R1966 = model.reactions.get_by_id('R1966')
R1966 #m1826[C_ex] -> it is removing extracellular erythritol
# Investigating reactions of erythriol_

# Find exchange reaction of erythriol_ 
m1826 = model.metabolites.get_by_id('m1826[C_ex]')
m1826 # in 2 reactions: R1963, R1966
# R1963
R1963 = list(m1826.reactions)[0]
R1963 # erythritol transport extracellular

Reaction identifier,R1966
Name,EXC_OUT_m1826
Memory address,0x7fc033e1ea90
Stoichiometry,m1826[C_ex] --> erythritol_ -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Find exchange reaction of D-Sorbitol
D-Sorbitol =?Find exchange R1073,D-sorbitol transport,True,"['D-glucitol_C6H14O6', 'D-glucitol_C6H14O6']","['m1199[C_ex]', 'm457[C_cy]']"


In [26]:
# transport reaction R1073
R1073 = model.reactions.get_by_id('R1073')
R1073 # metabolite id: m1199[C_ex] 
m1199 = list(R1073.metabolites.keys())[0]
m1199 # contributes to 2 reactions: R1073, R1068
# R1068
R1068 = list(m1199.reactions)[0]
R1068 # name: D-sorbitol transport (-> no exchange reaction exists)

Reaction identifier,R1073
Name,D-sorbitol transport
Memory address,0x7fc021215d60
Stoichiometry,m1199[C_ex] <=> m457[C_cy] D-glucitol_C6H14O6 <=> D-glucitol_C6H14O6
GPR,
Lower bound,-1000.0
Upper bound,1000.0


### understanding exchange reactions

In [27]:
# R1213,L-glutamine exchange,0.0,0.0
R1213 = model.reactions.get_by_id('R1213')
R1213.reversibility

False

In [28]:
# compare reaction R1070, D-glucose exchange and R851,glucose transport,False,"['D-glucose_C6H12O6', 'D-glucose_C6H12O6']","['m511[C_ex]', 'm295[C_cy]']"
# same metabolites?
R1070 = model.reactions.get_by_id('R1070')
print(R1070.metabolites)
R851 = model.reactions.get_by_id('R851')
print(R851.metabolites)
# yes same metabolites

{<Metabolite m511[C_ex] at 0x7fc020f1a190>: -1.0}
{<Metabolite m511[C_ex] at 0x7fc020f1a190>: -1.0, <Metabolite m295[C_cy] at 0x7fc001aa47f0>: 1.0}


### Transport reactions extracellular

In [29]:
# <groups:group sboTerm="SBO:0000633" groups:id="group95" groups:name="Transport, extracellular" groups:kind="partonomy">
transExcMembers = model.groups.group95.members

# generating dataframe with id, name and metabolites
extTransId = [reaction.id for reaction in transExcMembers]
extTransName = [reaction.name for reaction in transExcMembers]
extTransMetaboliteNames = [[metabolite.name for metabolite in list(reaction.metabolites.keys())] for reaction in transExcMembers]
extTransMetabolites = [[metabolite.id for metabolite in reaction.metabolites.keys()] for reaction in transExcMembers]
extTransReversibility = [reaction.reversibility for reaction in transExcMembers]
extTransZip = list(zip(extTransId,extTransName, extTransReversibility, extTransMetaboliteNames, extTransMetabolites))
extTransDf = pd.DataFrame(extTransZip, columns=['id', 'name', 'reversibility', 'metaboliteNames', 'metabolites'])
extTransDf.to_csv('./output/extracellular_transprot_reactions.csv', index=False)


## Generating table for usable carbon sources and their echange reactions
Load carbonSourceNames_textFromImage
Set up Pandas dataframe

In [30]:
# build script: cleanCarbonSourceNames.py

## Example of Guo et al. 2022 production 
### (hexanoate exchange R1439) (not workings)

In [31]:
# set glucose == 0
glu = model.reactions.get_by_id('R1070')
glu.bounds = (0,0)
# activate hexaExchange
hexaExchange = model.reactions.get_by_id('R1439')
hexaExchange.bounds = (-0.61,0)

# # set objective to citrate
# model.objective = "R1044"
# optimize + result

model.objective = "biomass_C"

res = model.optimize()
print(res)
# print(res.fluxes['R2277'])
# print(res.fluxes['R1439'])

model.medium

<Solution 0.031 at 0x7fc0097d3b20>


{'R1016': 1000.0,
 'R1030': 1000.0,
 'R1164': 1000.0,
 'R1189': 1000.0,
 'R1287': 1000.0,
 'R1296': 1000.0,
 'R1298': 1000.0,
 'R1323': 1000.0,
 'R1333': 1000.0,
 'R1364': 1000.0,
 'R1439': 0.61}

## Set up Sugarcane composition
- Todo:
    - Got Composition calculations
        - calculate growth rate (today)
    - Introduce pathways (cellulose, galactose)
    - Find products for theoretical yield calculation

In [51]:
#### Sugarcane bagasse
# set all the other influxes to zero (carbon sources)
# Get glucse flux bounds and set them to zero
glucoseReac = model.reactions.get_by_id('R1070')
print('glucose reaction bounds: ', glucoseReac.bounds) # they are null
glucoseReac.bounds = (0.0,0.0) # safety first
# 1. find carbon source exchange reactions with the table
# xylose: R1074, arabinose: R1062
# 2. Set the bounds to the corresponding values

# xylose: R1074
xyloseReac = model.reactions.get_by_id('R1074')
print('xylose reaction bounds: ', xyloseReac.bounds)
# set the lower bound to -0,49329
xyloseReac.bounds = (-0.49329, 0.0)
print('xylose reaction bounds: ', xyloseReac.bounds)
# arabinose: R1062
arabinoseReac = model.reactions.get_by_id('R1062')
print('arabinose reaction bounds: ', arabinoseReac.bounds)
# set the lower bound to -0,04374
arabinoseReac.bounds = (-0.04374, 0.0)
print('arabinose reaction bounds: ', arabinoseReac.bounds)

# 3. optimize and print resulting growth rate
sugarcaneBagasse = model.optimize() # optimized value 0.009

# 4. Reset the changed bounds again to 0
xyloseReac.bounds = (0.0,0.0)
arabinoseReac.bounds = (0.0,0.0)
sugarcaneBagasse
# what are the reduced_costs?

# fluxtable of bagasse
sugarcaneBagasse.fluxes.index.names = ['reactionId'] # set index column name
sugarcaneBagasse.fluxes.to_csv('output/testSugarcaneBagasse.csv')
sugarcaneBagasse


glucose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (-0.49329, 0.0)
arabinose reaction bounds:  (0.0, 0.0)
arabinose reaction bounds:  (-0.04374, 0.0)


Reaction identifier,biomass_C
Name,biomass_C
Memory address,0x7fc033bd84c0
Stoichiometry,0.035251 m1000[C_cy] + 0.172773 m1008[C_cy] + 0.943397 m1123[C_cy] + 0.567939 m114[C_cy] + 0.243866 m130[C_cy] + 0.235849 m1324[C_cy] + 23.09 m141[C_cy] + 0.059363 m149[C_cy] + 3.5e-05 m1631[C_em]... 0.035251 ergosterol_C28H44O + 0.172773 L-valine_C5H11NO2 + 0.943397 (1-3)-beta-D-glucan_ + 0.567939 L-alanine_C3H7NO2 + 0.243866 L-glutamine_C5H10N2O3 + 0.235849 mannan_ + 23.09 ATP_C10H16N5O13P3 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [33]:
### sugarcane straw
# set all the other influxes to zero (carbon sources)
# Get glucse flux bounds and set them to zero
glucoseReac = model.reactions.get_by_id('R1070')
glucoseReac.bounds = (0.0,0.0)
print('glucose reaction bounds: ', glucoseReac.bounds) # they are null

# 1. find carbon source exchange reactions with the table
# 2. Set the bounds to the corresponding values
# xylose: R1074, 
xyloseReac = model.reactions.get_by_id('R1074')
print('xylose reaction bounds: ', xyloseReac.bounds)
# set the lower bound to -0,497421
xyloseReac.bounds = (-0.497421, 0.0)
print('xylose reaction bounds: ', xyloseReac.bounds)
# arabinose: R1062
arabinoseReac = model.reactions.get_by_id('R1062')
print('arabinose reaction bounds: ', arabinoseReac.bounds)
# set the lower bound to -0,06245
arabinoseReac.bounds = (-0.06245, 0.0)
print('arabinose reaction bounds: ', arabinoseReac.bounds)

# 2. optimize and print resulting growth rate
sugarcaneStraw = model.optimize() # optimized value 0.011

# set changed bounds of reactions again to 0
xyloseReac.bounds = (0.0,0.0)
arabinoseReac.bounds = (0.0,0.0)
sugarcaneStraw
# what are the reduced_costs?

## fluxtable straw
sugarcaneStraw.fluxes.index.names = ['reactionId'] # set index column name
sugarcaneStraw.fluxes.to_csv('output/testSugarcaneStraw.csv')


glucose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (-0.497421, 0.0)
arabinose reaction bounds:  (0.0, 0.0)
arabinose reaction bounds:  (-0.06245, 0.0)


# Generation of iYli21_v2.xml

### Workflow: Introducing new reactions (like tutorial lycopene)
1. Add metabolites
2. Add reactions
3. Add genes (gene_reaction_rule (getting started (documentation)))

### Add the galactose pathway (4 genes)
0. Add alpha-D-galactose + exchange reaction
1. Add transport of alpha-D-galactose ex -> cy
2. Galactokinase
- Add metabolites: alpha-D-galactose, alpha-D-galactose-1-phosphate
- Add Reaction alpha-D-galactose + ATP => alpha-D-galactose-1-phosphate + ADP
3. [Galactose-1-phosphate uridyl transferase](https://www.uniprot.org/uniprotkb/Q6C0K3/entry) 
- Add metabolites alpha-D-galactose-1-phosphate, UDP-alpha-D-glucose, alpha-D-glucose-1-phosphate, UDP-alpha-D-galactose
- Add reaction (alpha-D-galactose-1-phosphate + UDP-alpha-D-glucose = alpha-D-glucose-1-phosphate + UDP-alpha-D-galactose)
4. [UDP-glucose-4-Epimerase](https://www.uniprot.org/uniprotkb/Q6C4H2/entry)
- Add metabolites UDP-alpha-D-glucose, UDP-alpha-D-galactose
- Add reaction UDP-alpha-D-glucose = UDP-alpha-D-galactose

5. add bounds 
6. simulate again
7. log the result



| Name         | Abbreviation     | Gene | id |
|--------------|-----------|------------|-------------|
| Galactokinase | ylGAL1 | YALI0C13090g | - |
| Galactose-1-phosphate uridyl transferase | ylGAL7  | YALI0F23947g       | - |
| UDP-glucose-4-Epimerase      | ylGAL10E  | YALI0E26829g       | - |
| Phosphoglucomutase     | ylGAL10M | YALI0C09570g       | R638 |

#### Found the metabolites implemented:
alpha-D-galactose, 
<species metaid="m544__91__C_cy__93__" id="m544__91__C_cy__93__" name="D-galactose_C6H12O6" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>


alpha-D-galactose-1-phosphate, C6H11O9P or 	C6H13O9P, , https://pubchem.ncbi.nlm.nih.gov/compound/alpha-D-Galactose-1-phosphate

```<species metaid="m545__91__C_cy__93__" id="m545__91__C_cy__93__" name="alpha-D-galactose 1-phosphate_C6H11O9P" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>```


UDP-alpha-D-glucose, C15H22N2O17P2, (exists),??not alpha,  https://pubchem.ncbi.nlm.nih.gov/compound/UDP-alpha-D-glucose_2

```<species metaid="m8__91__C_cy__93__" id="m8__91__C_cy__93__" name="UDP-D-glucose_C15H22N2O17P2" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>```

alpha-D-glucose-1-phosphate, C6H13O9P, (exists), https://pubchem.ncbi.nlm.nih.gov/compound/alpha-D-glucose-1-phosphate ,

```<species metaid="m589__91__C_cy__93__" id="m589__91__C_cy__93__" name="D-glucose 1-phosphate_C6H13O9P" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>```


UDP-alpha-D-galactose,

```<species metaid="m546__91__C_cy__93__" id="m546__91__C_cy__93__" name="UDP-D-galactose_C15H24N2O17P2" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>```


In [34]:
# Investigating existing reaction for Phosphoglucomutase
R638 = model.reactions.get_by_id('R638')
R638 # -> dont need to add it

Reaction identifier,R638
Name,phosphoglucomutase
Memory address,0x7fc021147df0
Stoichiometry,m296[C_cy] <=> m589[C_cy] D-glucose 6-phosphate_C6H13O9P <=> D-glucose 1-phosphate_C6H13O9P
GPR,YALI1E02616g
Lower bound,-1000.0
Upper bound,1000.0


#### 1. [Galactokinase](https://www.uniprot.org/uniprotkb/Q6CC28/entry)
alpha-D-galactose + ATP = ADP + alpha-D-galactose 1-phosphate + H+

In [35]:
# Galactokinase | ylGAL1 | YALI0C13090g
#### PLAN: 
# Introduce alpha-D-galactose (C_ex)
# Transport reaction -> alpha-D-galactose(C_cy)
# Add reaction with ATP cytosolic (m141[C_cy])

# pyruvate kinase (pyk) is an enzyme producing ATP -> Find ATP and also if it is extracellular available
# Search for ATP in model 
pyk = model.reactions.get_by_id('R694')
pyk # -> no extracellular ATP
# add this reaction (last reaction: R2285) (new suffix: _mccs)
# naming conventions Metabolites: (last metabolite: m2060)
    #first: <species metaid="m1__91__C_cy__93__" id="m1__91__C_cy__93__" name="(R)-lactate_C3H6O3" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    #last:  <species metaid="m2060__91__C_cy__93__" id="m2060__91__C_cy__93__" name="m2060[C_cy]" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>


# Create new metabolites

# alpha-D-galactose, 
# <species metaid="m544__91__C_cy__93__" id="m544__91__C_cy__93__" name="D-galactose_C6H12O6" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>


# alpha-D-galactose-1-phosphate, C6H11O9P or 	C6H13O9P, , https://pubchem.ncbi.nlm.nih.gov/compound/alpha-D-Galactose-1-phosphate
# <species metaid="m545__91__C_cy__93__" id="m545__91__C_cy__93__" name="alpha-D-galactose 1-phosphate_C6H11O9P" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>


# UDP-alpha-D-glucose, C15H22N2O17P2, (exists),??not alpha,  https://pubchem.ncbi.nlm.nih.gov/compound/UDP-alpha-D-glucose_2
# <species metaid="m8__91__C_cy__93__" id="m8__91__C_cy__93__" name="UDP-D-glucose_C15H22N2O17P2" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>

# alpha-D-glucose-1-phosphate, C6H13O9P, (exists), https://pubchem.ncbi.nlm.nih.gov/compound/alpha-D-glucose-1-phosphate , 
# <species metaid="m589__91__C_cy__93__" id="m589__91__C_cy__93__" name="D-glucose 1-phosphate_C6H13O9P" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>


# UDP-alpha-D-galactose,
# <species metaid="m546__91__C_cy__93__" id="m546__91__C_cy__93__" name="UDP-D-galactose_C15H24N2O17P2" compartment="C_cy" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>



from cobra import Reaction
# read v2 model:
model2 = read_sbml_model('./iYli21_v2.xml')

## Add exchange reaction 

reaction = Reaction('R_mccs_1') 
reaction.name = 'alpha-D-galactose exchange'
reaction.subsystem = 'Exchange' # don't used
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
dGalactose = model2.metabolites.get_by_id('m544[C_cy]')
reaction.add_metabolites({
    dGalactose: -1.0,
})
model2.add_reactions([reaction])

# is it ok, that my exchange reaction produces a d-galactose in the cytoplasm?
## add transport

# dGalactose_ex = ...
# 
# reaction = Reaction('R_mccs_2') 
# reaction.name = 'alpha-D-galactose transport'
# reaction.subsystem = 'Transport, extracellular' # don't used
# reaction.lower_bound = 0.  # This is the default
# reaction.upper_bound = 1000.  # This is the default
# dGalactose = model2.metabolites.get_by_id('m544[C_cy]')
# reaction.add_metabolites({
#     dGalactose_ex: -1.0,
#     dGalactose: 1.0,
# })
# model2.add_reactions([reaction])

# Take metabolites as they are
alphaDgalactose = model2.metabolites.get_by_id('m544[C_cy]')
alphaDgalactose1phosphate = model2.metabolites.get_by_id('m545[C_cy]')
UDPalphaDglucose = model2.metabolites.get_by_id('m8[C_cy]')
alphaDglucose1phosphate = model2.metabolites.get_by_id('m589[C_cy]')
UDPalphaDgalactose = model2.metabolites.get_by_id('m546[C_cy]')
ATP_cy = model2.metabolites.get_by_id('m141[C_cy]')
ADP_cy = model2.metabolites.get_by_id('m143[C_cy]')
h_plus = model2.metabolites.get_by_id('m10[C_cy]')

### Galactokinase
## Reaction alpha-D-galactose + ATP => alpha-D-galactose-1-phosphate + ADP
reaction = Reaction('R_mccs_3') 
reaction.name = 'Galactokinase'
reaction.subsystem = '' # don't used
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    alphaDgalactose: -1.0,
    ATP_cy: -1.0,
    ADP_cy: 1.0,
    alphaDgalactose1phosphate: 1.0,
    h_plus: 1.0
})
reaction.reaction
model2.add_reactions([reaction])

### Galactose-1-phosphate uridylyltransferase
# alpha-D-galactose 1-phosphate + UDP-alpha-D-glucose = alpha-D-glucose 1-phosphate + UDP-alpha-D-galactose

reaction = Reaction('R_mccs_4') 
reaction.name = 'Galactose-1-phosphate uridylyltransferase'
reaction.subsystem = '' # don't used
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    alphaDgalactose1phosphate: -1.0,
    UDPalphaDglucose: -1.0,
    alphaDglucose1phosphate: 1.0,
    UDPalphaDgalactose: 1.0,
})
reaction.reaction
model2.add_reactions([reaction])


### UDP-glucose-4-Epimerase
## UDP-alpha-D-glucose = UDP-alpha-D-galactose
reaction = Reaction('R_mccs_5') 
reaction.name = 'alpha-D-galactose exchange'
reaction.subsystem = 'Exchange' # don't used
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    UDPalphaDgalactose: -1.0,
    UDPalphaDglucose: 1.0,
})
reaction.reaction
model2.add_reactions([reaction])

# model2
### write sbml model
write_sbml_model(model2, "iYli21_v2.xml")

Ignoring reaction 'R_mccs_1' since it already exists.
Ignoring reaction 'R_mccs_3' since it already exists.
Ignoring reaction 'R_mccs_4' since it already exists.
Ignoring reaction 'R_mccs_5' since it already exists.


In [36]:
model2

Name,M_iYli21
Memory address,0x07fc0097e2610
Number of metabolites,1868
Number of reactions,2289
Number of groups,113
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"unknownCompartment1, unknownCompartment9, unknownCompartment3, unknownCompartment14, unknownCompartment4, unknownCompartment12, unknownCompartment11, unknownCompartment5, unknownCompartment2, unknownCompartment7, unknownCompartment10, unknownCompartment8, unknownCompartment6, unknownCompartment13, unknownCompartment15"


### Testing results of galactose pathway introduction (iYli21_v2)
#### Sugarcane bagasse

In [37]:
#### Sugarcane bagasse
# set all the other influxes to zero (carbon sources)
# Get glucse flux bounds and set them to zero
glucoseReac = model2.reactions.get_by_id('R1070')
print('glucose reaction bounds: ', glucoseReac.bounds) 
# set to null
glucoseReac.bounds = (0.0,0.0)
# 1. find carbon source exchange reactions with the table
# xylose: R1074, arabinose: R1062
# 2. Set the bounds to the corresponding values

# xylose: R1074
xyloseReac = model2.reactions.get_by_id('R1074')
print('xylose reaction bounds: ', xyloseReac.bounds)
# set the lower bound to -0,49329
xyloseReac.bounds = (-0.49329, 0.0)
print('xylose reaction bounds: ', xyloseReac.bounds)

# arabinose: R1062
arabinoseReac = model2.reactions.get_by_id('R1062')
print('arabinose reaction bounds: ', arabinoseReac.bounds)
# set the lower bound to -0,04374
arabinoseReac.bounds = (-0.04374, 0.0)
print('arabinose reaction bounds: ', arabinoseReac.bounds)

# galactose: R_mccs_1
galactoseReac = model2.reactions.get_by_id('R_mccs_1')
print('arabinose reaction bounds: ', galactoseReac.bounds)
# set the lower bound to -0,015795
galactoseReac.bounds = (-0.015795, 0.0)
print('arabinose reaction bounds: ', galactoseReac.bounds)

# 3. optimize and print resulting growth rate
sugarcaneBagasse = model2.optimize() # optimized value 0.011

# 4. Reset the changed bounds again to 0
xyloseReac.bounds = (0.0,0.0)
arabinoseReac.bounds = (0.0,0.0)
galactoseReac.bounds = (0.0,0.0)
sugarcaneBagasse

glucose reaction bounds:  (-2.43, 1000.0)
xylose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (-0.49329, 0.0)
arabinose reaction bounds:  (0.0, 0.0)
arabinose reaction bounds:  (-0.04374, 0.0)
arabinose reaction bounds:  (0.0, 1000.0)
arabinose reaction bounds:  (-0.015795, 0.0)


,fluxes,reduced_costs
R1,0.000000,-0.003374
R2,0.000000,-0.003374
R3,0.000000,-0.003374
R4,0.010413,0.000000
R5,0.000000,-0.300313
...,...,...
R2285,0.000000,0.000000
R_mccs_1,-0.015795,-0.276693
R_mccs_3,0.000000,0.000000
R_mccs_4,0.000000,0.000000


#### Sugarcane Straw

In [38]:
### sugarcane straw
# set all the other influxes to zero (carbon sources)
# Get glucse flux bounds and set them to zero
glucoseReac = model2.reactions.get_by_id('R1070')
glucoseReac.bounds = (0.0,0.0)
print('glucose reaction bounds: ', glucoseReac.bounds) # they are null

# 1. find carbon source exchange reactions with the table
# 2. Set the bounds to the corresponding values

# xylose: R1074, 
xyloseReac = model2.reactions.get_by_id('R1074')
print('xylose reaction bounds: ', xyloseReac.bounds)
# set the lower bound to -0,497421
xyloseReac.bounds = (-0.497421, 0.0)
print('xylose reaction bounds: ', xyloseReac.bounds)

# arabinose: R1062
arabinoseReac = model2.reactions.get_by_id('R1062')
print('arabinose reaction bounds: ', arabinoseReac.bounds)
# set the lower bound to -0,06245
arabinoseReac.bounds = (-0.06245, 0.0)
print('arabinose reaction bounds: ', arabinoseReac.bounds)

# galactose: R_mccs_1
galactoseReac = model2.reactions.get_by_id('R_mccs_1')
print('arabinose reaction bounds: ', galactoseReac.bounds)
# set the lower bound to -0,015795
galactoseReac.bounds = (-0.015795, 0.0)
print('arabinose reaction bounds: ', galactoseReac.bounds)

# 2. optimize and print resulting growth rate
sugarcaneStraw = model2.optimize() # optimized value 0.014

# set changed bounds of reactions again to 0
xyloseReac.bounds = (0.0,0.0)
arabinoseReac.bounds = (0.0,0.0)
sugarcaneStraw

glucose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (0.0, 0.0)
xylose reaction bounds:  (-0.497421, 0.0)
arabinose reaction bounds:  (0.0, 0.0)
arabinose reaction bounds:  (-0.06245, 0.0)
arabinose reaction bounds:  (0.0, 0.0)
arabinose reaction bounds:  (-0.015795, 0.0)


,fluxes,reduced_costs
R1,0.000000,-0.003374
R2,0.000000,-0.003374
R3,0.000000,-0.003374
R4,0.012897,0.000000
R5,0.000000,-0.300313
...,...,...
R2285,0.000000,0.000000
R_mccs_1,-0.015795,-0.276693
R_mccs_3,0.000000,0.000000
R_mccs_4,0.000000,0.000000


In [39]:
# what should be the compartment
# print the different compartments
compartmentList = [metabolite.compartment for metabolite in model.metabolites]
print(list(set(compartmentList)))
# ['C_mi', 'C_ex', 'C_gm', 'C_en', 'C_mm', 'C_cy', 'C_er', 'C_va', 'c_va', 'C_go', 'C_vm', 'C_pe', 'C_em', 'C_nu', 'C_lp']

['C_go', 'C_nu', 'C_en', 'C_er', 'C_ex', 'C_gm', 'c_va', 'C_em', 'C_mi', 'C_mm', 'C_vm', 'C_cy', 'C_pe', 'C_va', 'C_lp']


In [40]:
# check mass balance of all reactions:
allReactions = model.reactions
mbList = []
for reaction in allReactions:
    mb = reaction.check_mass_balance()
    if (mb != {}):
        print("Das habe ich nicht gehofft, es ist aber passiert.")
# -> Alle reactionene in mass balance

### Compartments of the model based on groups

|Abbreviation | Compartment name ? |
|-------------|------------------|
| C_cy | cytosol |
| C_em | ER membrane |
| C_en | cell envelope / envelope |
| C_er | Endoplasmic reticulum / ER |
| C_ex | Extracellular / extracellular |
| C_gm | Golgi membrane |
| C_go | Golgi / golgi |
| C_lp | lipid particle |
| C_mi | mitochondrion |
| C_mm | mitorchondrial membrane |
| C_nu | nucleus |
| C_pe | Peroxisomal / peroxisome |
| c_va |  |
| C_va | vacuolar / vacuole |
| C_vm | vacuolar membrane |



### Reactions of products

| reactionID | reactionName | metaboliteID_ext | product | class |
| -----------|--------------|------------------|---------|-------|
| R1044 | citrate transport | m1187__91__C_ex__93__ | citrate | organic acid |
| R1963 | erythritol transport extracellular | m1826__91__C_ex__93__ | Erythriol | sugar alcohols |
| R2279 | 3-dehydroshikimate transport | m2056__91__C_ex__93__ | 3-Dehydroshikimate | aromates |
| R2044 | fatty acid transport?? | m1885__91__C_ex__93__ | Oleic acid | usaturated fatty acid |
| R2277 | D-Limonene transport | m2055__91__C_ex__93__ |  Limonene | terpene |

## Find Pathway list and corresponding reactions


### build table to iterate over groups (names and ids)

In [41]:
# search for genes and reactions of Glyoxylate and dicarboxylate metabolism group
groupsList = model.groups
groupsList
# -> groups haben id, name und members
# 1. Table of group name and id
names = []
ids = []
for group in groupsList:
    names.append(group.name)
    ids.append(group.id)
zipped = list(zip(names, ids))

groupsTable = pd.DataFrame(zipped,columns=['groupName', 'id'])
groupsTable.to_csv('./output/groupsTable.csv')

### testing
nameOfInterest = 'Glyoxylate and dicarboxylate metabolism'

groupsTable.loc[groupsTable.groupName == nameOfInterest]
# -> 'Glyoxylate and dicarboxylate metabolism' has id: group38

,groupName,id
37,Glyoxylate and dicarboxylate metabolism,group38


## Interesting pathways (inspired by Guo et al. 2022)

Alkane metabolism, amino acid metabolism, citrate cycle (TCA cycle), Fatty acid metabolism, Glycerolipid metabolism, Glycolysis/Gluconeogenesis, Glyoxylate and dicarboxylate, Methane metabolism, Pentose phosphate pathway, Transport, Butanoate metabolism

In [42]:
# From Guo et al 2022 Fig. 4
interestingPathwayList = ["Alkane metabolism", "Amino acid metabolism", "Citrate cycle (TCA cycle)", "Fatty acid metabolism", "Glycerolipid metabolism", "Glycolysis/Gluconeogenesis", "Glyoxylate and dicarboxylate metabolism", "Methane metabolism", "Pentose phosphate pathway", "Transport", "Butanoate metabolism"]

# subset
interestingPathwaySubset = ["Amino acid metabolism", "Citrate cycle (TCA cycle)", "Exchange", "Fatty acid metabolism", "Glycerolipid metabolism", "Glycolysis/Gluconeogenesis", "Pentose phosphate pathway", "Transport"]

#### Create table from group and reaction information

In [43]:
### Help functions
# Generate group/pathway table

In [44]:
## Verifying that all interestingPathways are found
counter = 0
for member in model.groups:
    for pathway in interestingPathwaySubset:
        if(member.name == pathway):
            counter += 1

if (counter == len(interestingPathwaySubset)):
    print(f"All {counter} pathways have been found")

## build dict and than table
pathwayDict = {}
for member in model.groups:
    for pathway in interestingPathwaySubset:
        if(member.name == pathway):
            # get the reactions and store them in the dict
            reactionList = list(member.members)
            pathwayDict[pathway] = reactionList

# print(pathwayDict)


pathways = []
reactionIds = []
reactionNames = []
for pathwayName, reactionList in pathwayDict.items():
    for reaction in reactionList:
        pathways.append(pathwayName)
        reactionIds.append(reaction.id)
        reactionNames.append(reaction.name)

zipped = zip(pathways, reactionIds, reactionNames)
pathwayDf = pd.DataFrame(zipped, columns=["pathwayName", "reactionId", "reactionName"])
pathwayDf.to_csv("output/Group&PatwayTable.csv")
    


All 8 pathways have been found


In [45]:
### Look at the exchange reactions and their values:
exchange = pathwayDf[pathwayDf["pathwayName"] == "Exchange"]

# Iterate over the reactions and look at the bounds
reacBounds = [] 
for i in range(len(exchange)):
    reaction1 = exchange['reactionId'].loc[exchange.index[i]]
    reac1 = model2.reactions.get_by_id(reaction1)
    reacBounds.append(reac1.bounds)

exchange["bounds"] = reacBounds
exchange.to_csv("output/DefaultExchangeReactionBounds.csv", index=False)

/var/folders/16/hjc0t_h52537rmc_pnwtym_00000gn/T/ipykernel_11516/745151832.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exchange["bounds"] = reacBounds


In [46]:
## iterate over each group and generate a dict for the reactions: key: groupName -> ractions => "str" => "list"
groupDict = {}
for member in model.groups:
    groupDict[member.name] = list(member.members)

allReactionsInUniqueGroups = True

for groupName, reactionList in groupDict.items():
    for reaction in reactionList:
        for ggroupName, rreactionList in groupDict.items():
            if (ggroupName == groupName):
                continue
            for rreaction in rreactionList:
                if (reaction == rreaction):
                    allReactionsInUniqueGroups = False
                    print("Found a reaction in 2 groups:")
                    print(f"Reaction from group {groupName} was also found in {ggroupName}")
                    print(f"The reaction found is called {reaction} and is equal to {rreaction}")
    
if (allReactionsInUniqueGroups):
    print("All reactions are in unique groups")


# iterate over each group and generate a dict for the genes: key: groupName -> genes

groupGeneDict = {}
for member in model.groups:
    geneList = []
    reactionList = member.members
    for reaction in reactionList: 
        for gene in list(reaction.genes):
            geneList.append(gene.id)
    groupGeneDict[member.name] = geneList
allGenesInUniqueGroups = True
notUniqueGenes = []
for groupName, geneList in groupGeneDict.items():
    for gene in geneList:
        for ggroupName, ggeneList in groupGeneDict.items():
            if (ggroupName == groupName):
                continue
            for ggene in ggeneList:
                if (gene == ggene):
                    notUniqueGenes.append(gene)
                    allGenesInUniqueGroups = False
                    print("Found a Gene in 2 groups:")
                    print(f"Gene from group {groupName} was also found in {ggroupName}")
                    print(f"The Gene found is called {gene} and is equal to {ggene}")
    
if (allGenesInUniqueGroups):
    print("All Gemes are in unique groups")
else:
    print(f"Not all genes are only active in one group, there are {len(notUniqueGenes)/2} many genes which are found in at least 2 groups")


All reactions are in unique groups
Found a Gene in 2 groups:
Gene from group Alanine, aspartate and glutamate metabolism was also found in Arginine and proline metabolism
The Gene found is called YALI1B12993g and is equal to YALI1B12993g
Found a Gene in 2 groups:
Gene from group Alanine, aspartate and glutamate metabolism was also found in Arginine and proline metabolism
The Gene found is called YALI1B12993g and is equal to YALI1B12993g
Found a Gene in 2 groups:
Gene from group Alanine, aspartate and glutamate metabolism was also found in Arginine and proline metabolism
The Gene found is called YALI1B12993g and is equal to YALI1B12993g
Found a Gene in 2 groups:
Gene from group Alanine, aspartate and glutamate metabolism was also found in Glycine, serine and threonine metabolism
The Gene found is called YALI1E19852g and is equal to YALI1E19852g
Found a Gene in 2 groups:
Gene from group Alanine, aspartate and glutamate metabolism was also found in Glycine, serine and threonine metabolism

### get genes and reactions per group (member)

In [47]:
nameOfInterest = 'Glyoxylate and dicarboxylate metabolism'

# get the ID and the corresponding group
groupID = groupsTable.id.values[groupsTable.groupName == nameOfInterest]
groupOfInterestList = [i for i in groupsList if i.id == groupID]

# check the members and go through every reaction and gene and find if they are from different groups
grouOfInteres = groupOfInterestList[0]
grouOfInteres.members
number = 0
geneList = []
for reaction in grouOfInteres.members:
    print(f'{reaction} is influenced by {reaction.genes} genes')
    fGene = list(reaction.genes)[0]
    print(f'The first gene {fGene} influences {fGene} reactions')
    break
    # number += len(reaction.genes)
    # geneList.append(reaction.genes)

def getUniqueReactionsFromGroup(group):
    '''returns a list of reactions and genes which are the members of a given group'''
    membersList = group.members
    geneList

R194: 2.0 m301[C_cy] --> m109[C_cy] + 2.0 m32[C_cy] is influenced by frozenset({<Gene YALI1F38629g at 0x7fc033ada3a0>, <Gene YALI1E40671g at 0x7fc033ada340>, <Gene YALI1E41251g at 0x7fc033ada370>}) genes
The first gene YALI1F38629g influences YALI1F38629g reactions


### Check iNL895 maintenance again

In [9]:
import cobra
# import pfba
from cobra.flux_analysis import pfba
import yaml
config_name = 'model_config'
config_path = f'../config/{config_name}.yaml'

# load config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [10]:
# experimental data:
# all 
yli_glucose_uptake = [2.43, 0.64, 0.95, 0.33, 0.72, 0.98, 2.09] # from 2 different resources (xu)
yli_experimental_growth = [0.26, 0.048, 0.08, 0.03, 0.07, 0.1, 0.2]
# iNL895:
print('iNL895_model')
iNL895_model_path = config['models']['yliNL895']
iNL895_model = cobra.io.read_sbml_model(iNL895_model_path)

# set objective
iNL895_model.objective = 'biomass_C'

# maintenance: not existing/ not found


# Set glucose reaction to -2.43 mmol/h, 0.64 and 0.61 (based on Guo et al. 2022)
biomass_reaction = 'biomass_C' # 'biomass_C' # 'r_1814' # 'r_021_xxx'
glucose_exchange_reaction = 'r_51_exchange'
iNL895_simulation_growth = []
for condition in yli_glucose_uptake:
    with iNL895_model:
        dGlu = iNL895_model.reactions.get_by_id(glucose_exchange_reaction)
        dGlu.bounds = (-1000, condition)
        try:
            pfba_solution = pfba(iNL895_model)
            growthRate = pfba_solution.fluxes[biomass_reaction]
        except:
            growthRate = 0
        iNL895_simulation_growth.append(growthRate)
        print(f'Growth rate on {condition} mmol/h glucose: {growthRate}')


iNL895_model


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


Growth rate on 2.43 mmol/h glucose: 0.42610336523450126
Growth rate on 0.64 mmol/h glucose: 0.11222475462966304
Growth rate on 0.95 mmol/h glucose: 0.16658362015340586
Growth rate on 0.33 mmol/h glucose: 0.05786588910592006
Growth rate on 0.72 mmol/h glucose: 0.1262528489583709
Growth rate on 0.98 mmol/h glucose: 0.17184415552667148
Growth rate on 2.09 mmol/h glucose: 0.36648396433749325


In [18]:
# for rxn in iNL895_model.reactions:
#     print(rxn.name, rxn.id)

# # non-enzymatic reaction r_0765 (nothing interesting)
# r_0765 = iNL895_model.reactions.get_by_id('r_0765')
# r_0765


Reaction identifier,r_0765
Name,non-enzymatic reaction
Memory address,0x2a3fcfe20
Stoichiometry,s_0180 <=> s_0182 + s_0474 2-oxaloglutaric acid[mi] <=> 2-oxoadipic acid[mi] + carbon dioxide[mi]
GPR,
Lower bound,-1000.0
Upper bound,1000.0
